# Data Preprocessing

In [8]:
#Data Preprocessing

#Importing data set 1 
import pandas as pd
df = pd.read_csv("data_clinical_patient1.txt", sep='\t')
#print(df)
#print (len(df))
#print (df.head(1))
#print (df.size)
#print (df.shape)

#counting Na's in data set 1 
df.isnull()
df.isnull().sum().sum()
df.isnull().sum()

#characterizing data 
df.loc[df['Overall Survival Status']== "1:DECEASED", 'Overall Survival Status']= '1'
df.loc[df['Overall Survival Status']== "0:LIVING", 'Overall Survival Status']= '0'
df.drop([0,1,2,3],axis=0,inplace=True)

#changing column name 
df.rename(columns={"#Patient Identifier": "PatientID"},inplace=True)
df.rename(columns={"Overall Survival Status": "OS"},inplace=True)
df

#selecting columns to analyze 
nr= df.iloc[:, [0,30]] #columnas a analizar
nr

#importing Data Set 2 
df2 = pd.read_csv("data_RNA_Seq_v2_mRNA_median_Zscores1.txt", sep='\t')
#print (len(df2))
#print (df2.head(1))
#print (df2.size)
#print (df2.shape)

df2.drop('Entrez_Gene_Id', inplace=True,axis=1)
df2

#changing column name & remove NAs
print(df2.shape)
df2.dropna(subset=['Hugo_Symbol'],inplace=True)
df2.rename(columns={"Hugo_Symbol": "PatientID"},inplace=True)
print(df2.shape)
df2

#transposing data set 
df3 = df2.set_index('PatientID').T
df3

#pre-processing data set 
df3.index=df3.index.map(lambda x:str(x)[:-3])
df3.index

#Counting Na's in columns & estimating how many genes with complete column of missing values
df3.isnull().sum().sum()/177

#eliminating Na's 
print(df3.shape)
dfn=df3.dropna(1)
print(dfn.shape)
dfn

#merge data sets 
result=dfn.merge(nr,right_on='PatientID', left_index=True)
result=result.set_index('PatientID')
result

#selecting columns
X= result.drop(['OS'],axis=1)
Y= result.iloc[:,[-1]]

#finding data dimensions 
#print(X.shape)
#print(Y.shape)

Y = Y['OS']

(20531, 178)
(20502, 178)
(177, 20502)
(177, 20025)


In [9]:
#Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_predict

import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif

import sklearn
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import time 

# Feature Selection (ExtraTreesClassifier)

In [10]:
estimator = "ETC"

In [11]:
#Embedded Function
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

def extra_trees_classifier(n):
    fs= Pipeline([
        ("scaler", StandardScaler()),
        ("feature_select", SelectFromModel(ExtraTreesClassifier(n_estimators=n)) ),
    ])
    fit = fs.fit(X, Y)
    Xnew = fit.transform(X)
    Xnew_names = [col[0]  for col in zip(X.columns,fs.named_steps['feature_select'].get_support()) if col[1]]
    Xreduced = pd.DataFrame(Xnew,columns=Xnew_names)
    return Xreduced

In [12]:
#Start Timer
t0 = time.time()

#FILTER
Xred = extra_trees_classifier(n=100)
display(Xred)

#Stop Timer
t1 = time.time() - t0

#Calculate Elapsed Time
print("Time elapsed (seconds): ", t1) # CPU seconds elapsed (floating point)
print("Time elapsed (hours):", t1/3600)

,AACSL,AAGAB,ABCA1,ABCA2,ABCA3,ABCA7,ABCB10,ABCB11,ABCB1,ABCB5,...,ZNF792,ZNF814,ZNF826,ZNF841,ZNF8,ZNF878,ZNF99,ZSCAN5A,ZSWIM7,ZWINT
0,-0.246212,0.249300,-1.233603,-0.345799,0.229499,-0.384697,-0.614999,-0.453297,-0.1978,-0.183700,...,-0.543400,-0.679501,-0.179398,-0.871399,-1.078603,-0.864599,0.881010,0.008901,-0.055498,0.024597
1,-0.096512,0.884501,-0.978003,0.045501,0.018199,0.079901,-0.782299,-0.428697,-0.0870,-0.761700,...,0.433999,-0.009001,-0.046398,-0.219600,-0.771002,0.477102,-0.569066,-0.566397,-0.047398,0.358099
2,-0.452512,-0.453300,-0.587003,-0.180199,-0.677398,1.043398,-0.715299,-0.486197,-0.6257,-0.617100,...,-0.054201,0.291600,0.085802,3.478794,0.253399,-0.436599,-0.569066,-0.568497,-0.281899,0.832801
3,-0.054911,1.213401,-0.723803,-0.630399,-0.700198,1.155497,-0.569599,-0.406697,-0.6518,2.786898,...,-0.457200,1.645201,-0.577498,0.557699,1.442701,-1.579800,-0.569066,0.202201,-0.103298,0.540399
4,-0.452512,0.046000,-0.867503,-0.619599,-0.081700,-0.493396,-0.560499,-0.257297,0.0418,1.433299,...,-0.932500,-0.459701,-0.080598,-0.889299,-0.470002,-0.008398,-0.569066,0.118001,-0.091998,-0.199504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,0.051589,0.387800,0.899099,0.652501,0.235199,1.155997,-0.698499,-0.350297,0.1260,4.449598,...,0.294899,0.015199,0.515401,-0.436600,-0.091801,0.167702,1.202505,0.208801,-0.023498,-0.972207
173,-0.235212,-0.269900,-0.299203,0.090701,0.016999,0.174001,-0.262699,0.187902,-0.3130,-0.148501,...,-0.460000,-0.431301,-0.704698,-0.096000,-0.860703,-0.826399,-0.569066,-1.098396,-0.234099,-0.781606
174,3.252997,-0.912501,0.853999,-0.428599,-0.935498,-1.061794,-0.471099,-0.420997,-0.6721,-0.730200,...,-0.865300,-0.475701,-0.239898,0.516099,0.755500,-0.723399,0.299120,-1.197196,-0.285199,1.498003
175,-0.079511,1.017501,-1.243903,-0.154299,-0.171900,0.257501,0.464900,-0.567697,-0.6358,-0.757800,...,1.923999,-0.009801,-1.099797,0.249999,-0.042601,-0.071198,-0.569066,-0.920796,-0.271999,0.235498


Time elapsed (seconds):  2.2833728790283203
Time elapsed (hours): 0.0006342702441745334


In [ ]:
#Create CSV Name
k_features = len(Xred.columns)
csv_name = str("ANOVA_" + "k_" + str(k_features)+ ".csv")
print(csv_name)

#Save to ININ4998 Pancreatic Cancer Folder
display(Xred)
Xred.to_csv(csv_name)

# Classification Models

In [ ]:
#Extracting CSV File From Feature Selection (Manual Input)
X_csv = pd.read_csv(csv_name) #Example: 'ANOVA_k_100.csv'
X_FS = X_csv.set_index
display(X_FS)

In [ ]:
#Model 1: Logistic Regression
def createLogisticRegression():    
    logreg= Pipeline([
        ("scaler", StandardScaler()),
        ("logistic_reg", LogisticRegression(max_iter=500)),
    ])
    return logreg

#Model 2: SVC
from sklearn.ensemble import RandomForestClassifier
def createSVC():    
    pipe= Pipeline([
        ("scaler", StandardScaler()),
        ("classifier", SVC(decision_function_shape="ovo")),
    ])
    return pipe

#Model 3: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
def createRandomForestClassifier():    
    pipe= Pipeline([
        ("scaler", StandardScaler()),
        ("classifier", RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)),
    ])
    return pipe

In [ ]:
#Saving ROC Curves
def predict_with_data(data_x, data_y, classifier, figname=''):
    classifier.fit(data_x,data_y)
    y_scores = cross_val_predict(classifier, data_x, data_y, cv=10)
    scores = cross_val_score(classifier, data_x, data_y, scoring='accuracy', cv=10)
    
    #graph ROC curve 
    fpr, tpr, thresholds= roc_curve(data_y,y_scores)
    #save_plot_roc_curve(fpr, tpr, figname)
    
    return (precision_score(data_y, y_scores),recall_score(data_y, y_scores), 
            roc_auc_score(data_y,y_scores), accuracy_score(data_y,y_scores))

In [ ]:
#from sklearn.preprocessing import OrdinalEncoder
#ord_enc = OrdinalEncoder()
#code = ord_enc.fit_transform(code)
#Y.values
numres=pd.get_dummies(Y)
display(numres)

In [ ]:
#Logistic Regression Score
precision_log,recall_log,roc_log,acu_log=predict_with_data(X_FS,numres.loc[:,'DECEASED'],createLogisticRegression(),'_logreg')

print('Logistic Regression')
print('---------') 
print('Precision: ', precision_log) 
print('Recall: ', recall_log) 
print('ROC AUC Score: ', roc_log) 
print ('Accuracy: ', acu_log) 
print('---------')

In [ ]:
#SVC Score
precision_svc,recall_svc,roc_svc,acu_svc=predict_with_data(Xred,numres.loc[:,'DECEASED'],createSVC(),'_SVC')

print('SVC')
print('---------') 
print('Precision: ', precision_svc) 
print('Recall: ', recall_svc) 
print('ROC AUC Score: ', roc_svc) 
print ('Accuracy: ', acu_svc) 
print('---------')

In [ ]:
#Random Forest Classifier Score
precision_rf,recall_rf,roc_rf,acu_rf=predict_with_data(Xred,numres.loc[:,'DECEASED'],
                                                       createRandomForestClassifier(),'_randfor')

print('RandomForest')
print('---------') 
print('Precision: ', precision_rf) 
print('Recall: ', recall_rf) 
print('ROC AUC Score: ', roc_rf) 
print ('Accuracy: ', acu_rf) 
print('---------')